In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from csv import writer
from time import sleep

# Inițializează driverul Selenium
driver = webdriver.Chrome()
driver.get("https://www.ejobs.ro/locuri-de-munca")

accept_cookies_button = driver.find_element(By.XPATH, '//button[@data-test-id="accept-all-cookies-button"]')
accept_cookies_button.click()
sleep(2)  # Așteaptă puțin după acceptarea cookie-urilor

# Funcție pentru a derula pagina
def scroll_page():
    driver.execute_script("window.scrollBy(0, 1000);")
    sleep(2)  # Așteaptă pentru încărcarea joburilor noi

# Deschide fișierul CSV pentru scriere
with open('jobs.csv', 'w', newline='', encoding='utf-8') as f:
    thewriter = writer(f)
    header = ['Title', 'Company', 'Location', 'Experience']
    thewriter.writerow(header)

    # Numărul de derulări ale paginii pe care dorești să le faci
    num_scrolls = 10

    try:
        for _ in range(num_scrolls):
            scroll_page()

        while True:
            results = driver.find_elements(By.CLASS_NAME, 'JobCardWrapper')

            for result in results:
                try:
                    title = result.find_element(By.CSS_SELECTOR, 'h2.JCContentMiddle__Title').text.strip()
                    company = result.find_element(By.CLASS_NAME, 'JCContentMiddle__Info').text.strip()
                    location = result.find_element(By.CSS_SELECTOR, 'span.JCContentMiddle__Info').text.strip()

                    # Accesează link-ul fiecărui job și extrage nivelul de experiență
                    link_element = result.find_element(By.XPATH, './/a[contains(@href, "/user/locuri-de-munca/")]')
                    link_url = link_element.get_attribute("href")
                    driver.execute_script("window.open('" + link_url + "');")
                    driver.switch_to.window(driver.window_handles[1])
                    sleep(2)

                    try:
                        experience = driver.find_element(By.CLASS_NAME, 'JDSummary.JDSummary--BorderDouble').text.strip()
                    except NoSuchElementException:
                        experience = 'Not available'

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    sleep(2)

                    job_info = [title, company, location, experience]
                    print(job_info)
                    thewriter.writerow(job_info)

                except NoSuchElementException as e:
                    print(f"An error occurred: {str(e)}")
                    continue

            next_button = driver.find_element(By.XPATH, '//a[@class="JLPButton JLPButton--Next"]')
            driver.execute_script("arguments[0].click();", next_button)
            sleep(4)  # Așteaptă 4 secunde pentru încărcarea completă a paginii și elementelor

    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Închide browserul
driver.quit()


['Inginer proiectare geotehnică', 'Skilld by eJobs', 'București', 'Mid-Level (2-5 Ani)']
['Lucrator Comercial', 'Skilld', 'București, Otopeni', 'Entry-Level (< 2 Ani)']
['Game Show Host 4,000 RON net + 400 RON meal tickets + BONUS 500 RON', 'EZUGI', 'București', 'Fără Experiență, Entry-Level (< 2 Ani) și alte 1']
['System Administrator', 'Macarne LLC', 'Remote (de acasă)', 'Mid-Level (2-5 Ani), Senior-Level (> 5 Ani)']
['Application Engineer (m/f/d)', 'MOTUL Deutschland GmbH', 'București', 'Mid-Level (2-5 Ani), Senior-Level (> 5 Ani)']
['Tehnician Agronom Cercetare (m/f/d)', 'KWS Group', 'Alexandria', 'Mid-Level (2-5 Ani)']
['Customer Service Representative - Maternity Leave cover', 'Mondelez', 'București', 'Mid-Level (2-5 Ani)']
['German Speaking Customer Service Advisor -Relocate to Greece', 'Webhelp Greece', 'Remote (de acasă)', 'Entry-Level (< 2 Ani)']
['Game Presenter Bucuresti, in limba Engleza: 3400 RON Net + BONUS', 'Evolution Romania', 'București, Ilfov, Chitila', 'Fără Experi